In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import regex as re
import geocoder
from geopy.geocoders import Nominatim
import geopy.distance

In [3]:
# Reading in both sets of scraped data
# scraped_data is from data scraped newest first, scraped_data2 is from data scraped oldest first
# As there was around 2000 data entried and I could scrape 1000 at a time, this should have got me
# most properties available in the newcaslte area

dfscrap = pd.read_csv('scraped_data.csv')
dfscrap2 = pd.read_csv('scraped_data2.csv')
dfscrap3 = pd.read_csv('scraped_data3.csv')
dfscrap4 = pd.read_csv('scraped_data4.csv')
dfscrap5 = pd.read_csv('scraped_data5.csv')

In [4]:
# Concatenating both dataframes
df1 = pd.concat([dfscrap,dfscrap2,dfscrap3,dfscrap4,dfscrap5])

In [5]:
# Resetting the index and dropping the index column
df1.reset_index(inplace = True)
df1.drop(columns='index', inplace=True)

In [6]:
# Dropping the Unnamed:0 column, as this will interfere when I test for duplicates
df1.drop(columns = ['Unnamed: 0'], inplace = True)

In [7]:
## Checking for duplicates
# It isn't so surprising to have so many duplicate properties
# On the Rightmove site they have multiple 'hot' properties which show up on random pages
df1.duplicated().sum()

1445

In [8]:
### This is the list of duplicate properties
dfdupe = df1[df1.duplicated(keep = False)].sort_values('address')
dfdupe

,price,address,house_type,bedroom
2057,"£218,995Knowing the purchase price means you c...",\t\r\nHylton Road\r\nAcklam\r\nMiddlesbrough\r...,Detached,×4
2710,"£132,995Knowing the purchase price means you c...",\t\r\nHylton Road\r\nAcklam\r\nMiddlesbrough\r...,Semi-Detached,×3
2709,"£132,995Knowing the purchase price means you c...",\t\r\nHylton Road\r\nAcklam\r\nMiddlesbrough\r...,Semi-Detached,×3
3690,"£181,995Knowing the purchase price means you c...",\t\r\nHylton Road\r\nAcklam\r\nMiddlesbrough\r...,Detached,×3
2060,"£116,995Knowing the purchase price means you c...",\t\r\nHylton Road\r\nAcklam\r\nMiddlesbrough\r...,Semi-Detached,×2
...,...,...,...,...
2044,"£302,995Knowing the purchase price means you c...","Yarm Back Lane,\r\nStockton-On-Tees,\r\nTS21 1AU",Detached,×4
757,"£295,995Knowing the purchase price means you c...","to the West of Sage and East of Dinnington,\r\...",Semi-Detached,×4
1811,"£228,995Knowing the purchase price means you c...","to the West of Sage and East of Dinnington,\r\...",Semi-Detached,×3
97,"£295,995Knowing the purchase price means you c...","to the West of Sage and East of Dinnington,\r\...",Semi-Detached,×4


In [9]:
### Removing all duplicate properties
df1.drop_duplicates(inplace = True)

In [10]:
### Giving each property a property number
df1['property_no'] = df1.index +1

In [11]:
df1

,price,address,house_type,bedroom,property_no
0,"£196,000Knowing the purchase price means you c...","Greystoke,\r\nPonteland,\r\nNE20 0AL",Semi-Detached,×3,1
1,"£389,995Knowing the purchase price means you c...",120 High Street\r\nGosforth\r\nNewcastle Upon ...,Detached,×4,2
2,"£350,000Knowing the purchase price means you c...","6 Cheviot House,\r\nManor Walks,\r\nCramlingto...",Detached,×4,3
3,"£200,000Knowing the purchase price means you c...","582 Durham Road,\r\nLow Fell,\r\nGateshead,\r\...",Semi-Detached,×3,4
4,"£335,995Knowing the purchase price means you c...","Off Beacon Lane,\r\nCramlington,\r\nNE23 8AZ",House,×4,5
...,...,...,...,...,...
5011,"£150,000Knowing the purchase price means you c...","Unit K The Galleries,\r\nWashington Centre,\r\...",Semi-Detached,×2,5012
5013,"£255,000Knowing the purchase price means you c...","First Floor, \r\n50 St. Mary Axe,\r\nLondon,\r...",Detached,×4,5014
5014,"£180,000Knowing the purchase price means you c...","48-49 The Galleries Washington Centre, Washing...",Town House,×4,5015
5015,"£299,950Knowing the purchase price means you c...","64 Sea Road,\r\nFulwell,\r\nSunderland,\r\nSR6...",Semi-Detached Bungalow,×2,5016


In [12]:
### Checking for nulls and datatypes of my columns
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3573 entries, 0 to 5016
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   price        3573 non-null   object
 1   address      3573 non-null   object
 2   house_type   3573 non-null   object
 3   bedroom      3573 non-null   object
 4   property_no  3573 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 167.5+ KB


<font size="6"><font color='orange'>Start of data cleaning on the 'price' column</font>

In [13]:
## Removing everything except for the numbers in my price column
df1['price'] = [re.sub('Knowing the purchase price means you can work out the total cost of buying the property.Read more about property price in our glossary page.','', x) for x in df1['price']]
df1['price'] = [re.sub(',','', x) for x in df1['price']]
df1['price'] = [re.sub('£','', x) for x in df1['price']]

In [14]:
## Removing all rows where the value is 'coming soon'
df1 = df1[df1['price'] != 'Coming Soon']

In [15]:
## Setting the price column to an integer type
df1['price'] = df1['price'].astype('int')

C:\Users\wdre6\AppData\Local\Temp\ipykernel_13760\3667124339.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['price'] = df1['price'].astype('int')


In [16]:
## Double-checking each unique value within the price column to see if anything else needs removing
df1['price'].value_counts()

125000    72
90000     68
120000    67
150000    67
110000    63
          ..
352995     1
139999     1
404995     1
338995     1
499000     1
Name: price, Length: 600, dtype: int64

<font size="6"><font color='orange'>Start of data cleaning on the 'address' column</font>

In [17]:
## remove the 'Nationwide' value
df1 = df1[df1['address'] != 'Nationwide']

In [18]:
## Replacing the /r/n value with a lot of spaces
# This will allow me to more easily isolate the postcode
df1['address'] = [re.sub('\r\n','      ', x) for x in df1['address']]

In [19]:
## Stripping the data and removing commas
# Stripping again to ensure there is no white space left
def clean_add (df):

    df = df.strip()
    df = df.replace(',','')
    df = df.strip()
    return df

In [20]:
## Applying the function
df1['address'] = df1['address'].apply(clean_add)

In [21]:
## Removing everything except for the last 8 characters, as these 100% contain the postcode
def postcode (x):
    return x[-8:]

In [22]:
## Applying the function
df1['address'] = df1['address'].apply(postcode)

In [23]:
## I know there are spaces between the first part and second part of the postcode, so I replace the spaces with commas
def commasplit (df):

    df = df.strip()
    df = df.replace(' ',',')

    return df

In [24]:
## Applying the function
df1['address'] = df1['address'].apply(commasplit)

In [25]:
## Creating a new column 'postcode' from the 1st split of the address column
# 'Postcode' will contain the first part of each postcode
df1['postcode'] = df1['address'].str.split(',').str[0]

In [26]:
df1['postcode'].value_counts()

NE26    189
NE9     170
SR1     152
NE3     141
TS18    140
       ... 
DL8       1
SL9       1
W1U       1
DL7       1
BH8       1
Name: postcode, Length: 122, dtype: int64

In [27]:
## Removing anything else left in the postcode column
df1 = df1[df1['postcode'] != 'r']
df1 = df1[df1['postcode'] != 'yne']

In [28]:
# Dropping the leftover 'address' column
df1.drop(columns = ['address'], inplace = True)

In [29]:
## Removing all postcodes not in Newcastle or the surrounding area
## This is any postcode not marked NE, SR or DH

remove_postcode = ['DH4','B90','DH1','WC2N','W6','M32','WD18','DH3',
                   'Teesside','DH5','EC3A','LS9','e','LS22','LS7','DL3',
                   'M17','AL1','PE9','DL14','BD19','NG17','M50','DL1',
                   'RM14','EC1M','BN12','DH9','B3','M34','LS1','CH1','LE1','W1G','LS8','L12',
                   'Scotland','RM11','NG4','M3','W1K','DL7','SS9','KT1','WV1','DL12',
                   'E16','WD17','DN10','RH10','CR2','DL8','SL9','W1U','BH8','S75']


In [30]:
## Removing all postcodes not in the Newcastle area
df1=df1[~df1['postcode'].isin(remove_postcode)]

In [31]:
## Checking the unique values left in the postcode column
df1['postcode'].value_counts() #.reset_index().sort_values(by = 'index')

NE26    189
NE9     170
SR1     152
NE3     141
TS18    140
       ... 
NE11      2
TS3       1
NE46      1
TS19      1
NE63      1
Name: postcode, Length: 64, dtype: int64

<font size="6"><font color='orange'>Converting the postcode to longitude and latitude</font>

In [32]:
## List of postcodes to get lat_long from

# Postcode list from dataframe (NE)
pc_ne_list = ['NE26','NE9','NE3','NE5','NE13','NE20','NE40','NE28','NE33','NE2',
           'NE16','NE12','NE23','NE30','NE31','NE32','NE4','NE24','NE25','NE15',
           'NE29','NE7','NE42','NE21','NE39','NE38','NE1','NE6','NE45','NE61',
           'NE22','NE36','NE8','NE11','NE46','NE63']

# Specific postcodes in order to get the longitude and latitude (NE)
pc_ne_list_spec = ['NE261AD','NE95AD','NE31AD','NE51AD','NE136AD','NE200AD','NE404AD','NE280AD','NE331AF','NE21AD',
                'NE163AD','NE126DD','NE231WW','NE301AD','NE311AD','NE323AA','NE45AD','NE241AA','NE250AD','NE150AA',
                'NE290AA','NE77AA','NE425AD','NE214AB','NE391AB','NE380AB','NE11AD','NE61AD','NE455AA','NE611AA',
                'NE225AA','NE360AD','NE81AD','NE110AD','NE461AB','NE630AE']

# Postcode list from dataframe (SU)
pc_su_list = ['SR1','SR6','SR2','SR3','SR7','SR8']

# Specific postcodes in order to get the longitude and latitude (SU)
pc_su_list_spec = ['SR11SA','SR60AA','SR27QT','SR31SA','SR70AE','SR81AA']

# Postcode list from dataframe (MI)
pc_mi_list = ['TS18','TS1','TS7','TS10','TS6','TS14','TS23','TS5',
              'TS15','TS26','TS17','TS24','TS9','TS8','TS20',
              'TS22','TS12','TS21','TS16','TS4','TS3','TS19']

# Specific postcodes in order to get the longitude and latitude (MI)
pc_mi_list_spec = ['TS181AA','TS11AA','TS70EF','TS101AB','TS60AB','TS146ab','TS231AB','TS54AB',
                    'TS150AA','TS260AA','TS170AA','TS240AE','TS95AA','TS80AA','TS201AA',
                    'TS225AA','TS121AB','TS211AA','TS160AA','TS42AF','TS30AA','TS190AA']

In [33]:
## Using geocoder to get the longitude and latitude for each postcode
# Appending these to lists in order to create a dataframe

# calling the Nominatim tool
loc = Nominatim(user_agent="GetLoc")

longitude_ne = []
latitude_ne = []
longitude_su = []
latitude_su = []
longitude_mi = []
latitude_mi = []

for i in pc_ne_list_spec:

    # Looping through the postocde list and getting the location lat and long
    try:
        getLoc = loc.geocode(i)
    except Exception as e:
        print(i)

    longitude_ne.append(getLoc.longitude)
    latitude_ne.append(getLoc.latitude)

for i in pc_su_list_spec:

    # Looping through the postocde list and getting the location lat and long
    try:
        getLoc = loc.geocode(i)
    except Exception as e:
        print(i)

    longitude_su.append(getLoc.longitude)
    latitude_su.append(getLoc.latitude)

for i in pc_mi_list_spec:

    # Looping through the postocde list and getting the location lat and long
    try:
        getLoc = loc.geocode(i)
    except Exception as e:
        print(i)

    try:
        longitude_mi.append(getLoc.longitude)
        latitude_mi.append(getLoc.latitude)
    except Exception as e:
        print(i)

In [34]:
## Creating a dictionary with the postcode, latitude and longitude
dict_ne_coor = {'postcode': pc_ne_list,'latitude': latitude_ne,'longitude':longitude_ne}

## Creating a dataframe to merge with df1
mapping_ne_df = pd.DataFrame(dict_ne_coor)

In [35]:
## Creating a dictionary with the postcode, latitude and longitude
dict_su_coor = {'postcode': pc_su_list,'latitude': latitude_su,'longitude':longitude_su}

## Creating a dataframe to merge with df1
mapping_su_df = pd.DataFrame(dict_su_coor)

In [36]:
## Creating a dictionary with the postcode, latitude and longitude
dict_mi_coor = {'postcode': pc_mi_list,'latitude': latitude_mi,'longitude':longitude_mi}

## Creating a dataframe to merge with df1
mapping_mi_df = pd.DataFrame(dict_mi_coor)

In [37]:
## Function to loop through the longitude and latitude columns and calculate the distance from them
# to the centre_coor (centre_coor is the long and lat of the town centre)

def pc_ne_dist(lat,long):

    centre_coor = (54.967517, -1.615484)

    return (geopy.distance.geodesic(centre_coor, (lat,long)).km)

In [38]:
## Function to loop through the longitude and latitude columns and calculate the distance from them
# to the centre_coor (centre_coor is the long and lat of the town centre)

def pc_su_dist(lat,long):

    centre_coor = (54.908542, -1.379728)

    return (geopy.distance.geodesic(centre_coor, (lat,long)).km)

In [39]:
## Function to loop through the longitude and latitude columns and calculate the distance from them
# to the centre_coor (centre_coor is the long and lat of the town centre)

def pc_mi_dist(lat,long):

    centre_coor = (54.575852, -1.244942)

    return (geopy.distance.geodesic(centre_coor, (lat,long)).km)

In [40]:
### ## Applying the function to my df1 to create a 'distance' column
# merging each dataframe together
mapping_ne_df['distance'] = mapping_ne_df[['latitude','longitude']].apply(lambda x: pc_ne_dist(x['latitude'],x['longitude']), axis = 1)
mapping_su_df['distance'] = mapping_su_df[['latitude','longitude']].apply(lambda x: pc_su_dist(x['latitude'],x['longitude']), axis = 1)
mapping_mi_df['distance'] = mapping_mi_df[['latitude','longitude']].apply(lambda x: pc_mi_dist(x['latitude'],x['longitude']), axis = 1)

mapping_df = pd.concat([mapping_ne_df,mapping_su_df,mapping_mi_df])

In [41]:
## Merging the mapping_df with the original df1 to give each postcode a long and lat
df1 = df1.merge(mapping_df, how = 'left', on = 'postcode')

In [42]:
df1

,price,house_type,bedroom,property_no,postcode,latitude,longitude,distance
0,196000,Semi-Detached,×3,1,NE20,55.06467,-1.75250,13.920984
1,389995,Detached,×4,2,NE3,55.00622,-1.61665,4.309192
2,350000,Detached,×4,3,NE23,55.09966,-1.59675,14.759397
3,200000,Semi-Detached,×3,4,NE9,54.94586,-1.59928,2.624904
4,335995,House,×4,5,NE23,55.09966,-1.59675,14.759397
...,...,...,...,...,...,...,...,...
3027,20000,Maisonette,×2,5010,NE38,54.90152,-1.55234,8.388036
3028,95000,Terraced,×2,5011,SR1,54.90546,-1.38168,0.365226
3029,150000,Semi-Detached,×2,5012,NE38,54.90152,-1.55234,8.388036
3030,180000,Town House,×4,5015,NE38,54.90152,-1.55234,8.388036


<font size="6"><font color='orange'>Start of data cleaning on the 'house_type' column</font>

In [56]:
## Removing the values that were scraped incorrectly (these x values refer to the bedroom number I believe)
## Removing the 'plot', as this isn't a house
## Removing the 'lodge', as these aren't brick buildings so may mess with the model
## Removing the 'Retirement Property', as this isn't a house
df1 = df1[df1['house_type'] != '×4']
df1 = df1[df1['house_type'] != '×2']
df1 = df1[df1['house_type'] != '×3']
df1 = df1[df1['house_type'] != '×9']
df1 = df1[df1['house_type'] != '×22']
df1 = df1[df1['house_type'] != 'Plot']
df1 = df1[df1['house_type'] != 'Lodge']
df1 = df1[df1['house_type'] != 'Retirement Property']
df1 = df1[df1['house_type'] != 'Flat']
df1 = df1[df1['house_type'] != 'Bungalow']
df1 = df1[df1['house_type'] != 'Apartment']
df1 = df1[df1['house_type'] != 'Detached Bungalow']
df1 = df1[df1['house_type'] != 'Semi-Detached Bungalow']
df1 = df1[df1['house_type'] != 'Ground Flat']
df1 = df1[df1['house_type'] != 'Terraced Bungalow']
df1 = df1[df1['house_type'] != 'Park Home']
df1 = df1[df1['house_type'] != 'Land']
df1 = df1[df1['house_type'] != 'Block of Apartments']
df1 = df1[df1['house_type'] != '2,982 sq. ft.(277 sq. m.)']
df1 = df1[df1['house_type'] != 'Semi-Detached Share']

In [44]:
## Changing the two 'Mews' properties to what house type they actually are
df1.loc[848, 'house_type'] = 'Terraced'
df1.loc[870, 'house_type'] = 'Semi-Detached'

In [45]:
df1[df1['house_type'] == 'Mews']

,price,house_type,bedroom,property_no,postcode,latitude,longitude,distance


In [46]:
## I double-checked what house type best fits each of these properties changing everything to either a;
# 'Detached', 'Semi-Detached' or 'Terraced' house
def clean_house (df):

    df = df.replace('Stone House', 'Detached')
    df = df.replace('Character Property', 'Semi-Detached')
    df = df.replace('Cottage', 'Detached')
    df = df.replace('Barn Conversion', 'Detached')
    df = df.replace('Link Detached House', 'Detached')
    df = df.replace('Town House', 'Semi-Detached')
    df = df.replace('End of Terrace', 'Semi-Detached')
    df = df.replace('House', 'Semi-Detached')
    df = df.replace('Farm Semi-Detached', 'Semi-Detached')
    df = df.replace('Country Semi-Detached', 'Semi-Detached')
    df = df.replace('End of Terrace', 'Semi-Detached')
    df = df.replace('House', 'Semi-Detached')
    df = df.replace('Maisonette', 'Terraced')
    df = df.replace('Detached Villa', 'Detached')

    return df

In [47]:
## Applying the function
df1['house_type'] = df1['house_type'].apply(clean_house)

In [57]:
## Checking the unique values left in the house_type column
df1['house_type'].value_counts()

Semi-Detached    1298
Detached          750
Terraced          623
Name: house_type, dtype: int64

<font size="6"><font color='orange'>Start of data cleaning on the 'bedroom' column</font>

In [49]:
## This function removes the '×' character
def clean_bed (df):

    df = df.replace('×','')

    return df

In [50]:
## Applying the function
df1['bedroom'] = df1['bedroom'].apply(clean_bed)

In [51]:
## Removing any properties that have 'Ask agent' or 'Freehold' in this column as they just were not
# successfully web scraped
df1 = df1[df1['bedroom'] != 'Ask agent']
df1 = df1[df1['bedroom'] != 'Freehold']
df1 = df1[df1['bedroom'] != '11']
df1 = df1[df1['bedroom'] != '12']
df1 = df1[df1['bedroom'] != '22']
df1 = df1[df1['bedroom'] != '26']

In [52]:
## Changing the bedroom column to an integer type
df1['bedroom'] = df1['bedroom'].astype('int')

In [53]:
## Checking the unique values left in the bedroom column
df1['bedroom'].value_counts()

3    1290
4     678
2     461
5     163
6      42
1      22
7       9
8       5
9       2
Name: bedroom, dtype: int64

In [54]:
df1.describe()

,price,bedroom,property_no,latitude,longitude,distance
count,2.672000e+03,2672.000000,2672.000000,2672.000000,2672.000000,2672.000000
mean,2.533974e+05,3.261228,1881.664296,54.878393,-1.473568,7.604685
std,2.257823e+05,0.951002,1346.190647,0.193532,0.190438,5.041167
min,2.000000e+04,1.000000,1.000000,54.454270,-2.088820,0.000048
25%,1.300000e+05,3.000000,806.750000,54.682480,-1.616650,3.889935
50%,2.000000e+05,3.000000,1657.500000,54.960530,-1.487093,7.048287
75%,3.000000e+05,4.000000,2597.250000,55.006220,-1.341780,12.167572
max,3.950000e+06,9.000000,5015.000000,55.182780,-0.973980,30.314963


<font size="6"><font color='orange'>Save the clean data</font>

In [58]:
## Converting this cleaned data to a cvs so I can open it within a different notebook to perform EDA
# I feel its cleaner this way
df1.to_csv('cleaned_up_data.csv')